In [10]:
# Computing tanh(x(t).dot(W) + b) elementwise

import theano
import theano.tensor as T
import numpy as np

X = T.matrix('X')
W = T.matrix('W')
b_sym = T.vector('b_sym')

results, updates = theano.scan(lambda v: T.tanh(T.dot(v, W) + b_sym), sequences=X)
compute_elementwise = theano.function(inputs=[X, W, b_sym], outputs=[results])

x = np.eye(2, dtype=theano.config.floatX)
w = np.ones((2, 2), dtype=theano.config.floatX)
b = np.ones((2), dtype=theano.config.floatX)
b[1] = 2

print compute_elementwise(x, w, b)[0]
print np.tanh(x.dot(w) + b)

[[ 0.96402758  0.99505475]
 [ 0.96402758  0.99505475]]
[[ 0.96402758  0.99505475]
 [ 0.96402758  0.99505475]]


In [43]:
# Computing the sequence x(t) = tanh(x(t - 1).dot(W) + y(t).dot(U) + p(T - t).dot(V))

X = T.vector('X')
W = T.matrix('W')
b_sym = T.vector('b_sym')

U = T.matrix('U')
Y = T.matrix('Y')
V = T.matrix('V')
P = T.matrix('P')

results, updates = theano.scan(lambda y, p, x_r: T.tanh(T.dot(x_r, W) + T.dot(y, U) + T.dot(p, V)),
                              sequences=[Y, P[::-1]], outputs_info=X)
compute_seq = theano.function(inputs=[X, W, Y, U, P, V], outputs=results)

x = np.zeros((2), dtype=theano.config.floatX)
x[1] = 1

w = np.ones((2, 2), dtype=theano.config.floatX)
y = np.ones((5, 2), dtype=theano.config.floatX)
y[0, :] = -3
u = np.ones((2, 2), dtype=theano.config.floatX)
p = np.ones((5, 2), dtype=theano.config.floatX)
p[0, :] = -3
v = np.ones((2, 2), dtype=theano.config.floatX)

print compute_seq(x, w, y, u, p, v)

x_res = np.zeros((5, 2), dtype=theano.config.floatX)
x_res[0] = np.tanh(x.dot(w) + y[0].dot(u) + p[4].dot(v))
print 
for i in range(1, 5):
    x_res[i] = np.tanh(x_res[i-1].dot(w) + y[i].dot(u) + p[4-i].dot(v))       
print x_res

[[-0.99505475 -0.99505475]
 [ 0.96471973  0.96471973]
 [ 0.99998585  0.99998585]
 [ 0.99998771  0.99998771]
 [-0.96402932 -0.96402932]]

[[-0.99505475 -0.99505475]
 [ 0.96471973  0.96471973]
 [ 0.99998585  0.99998585]
 [ 0.99998771  0.99998771]
 [-0.96402932 -0.96402932]]


In [51]:
X = T.matrix('X')
results, updates = theano.scan(lambda x_i: T.sqrt((x_i**2).sum()), sequences=X)
norm_func = theano.function([X], results)

x = np.diag(np.arange(1, 6, dtype = theano.config.floatX), 1)
print norm_func(x)

print np.sqrt((x**2).sum(1))

[ 1.  2.  3.  4.  5.  0.]
[ 1.  2.  3.  4.  5.  0.]


In [63]:
X = T.matrix('X')
results, updates = theano.scan(lambda x_i: T.sqrt((x_i**2).sum()), sequences=X.T)
norm_func = theano.function([X], results)

x_t_func = theano.function([X], X.T)

x = np.diag(np.arange(1, 6, dtype = theano.config.floatX), 1)

print x

res = norm_func(x)
print res
tr = x_t_func([res, res, res])
print tr
print x_t_func(tr)
print np.sqrt((x**2).sum(0))

[[ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.  2.  0.  0.  0.]
 [ 0.  0.  0.  3.  0.  0.]
 [ 0.  0.  0.  0.  4.  0.]
 [ 0.  0.  0.  0.  0.  5.]
 [ 0.  0.  0.  0.  0.  0.]]
[ 0.  1.  2.  3.  4.  5.]
[[ 0.  0.  0.]
 [ 1.  1.  1.]
 [ 2.  2.  2.]
 [ 3.  3.  3.]
 [ 4.  4.  4.]
 [ 5.  5.  5.]]
[[ 0.  1.  2.  3.  4.  5.]
 [ 0.  1.  2.  3.  4.  5.]
 [ 0.  1.  2.  3.  4.  5.]]
[ 0.  1.  2.  3.  4.  5.]
